In [ ]:
%cd ..
%cd mmdetection

## Download models

link: https://github.com/open-mmlab/mmdetection/tree/main/configs


In [ ]:
# We download the pre-trained checkpoints for inference and finetuning.
!mkdir ./checkpoints
!mim download mmdet --config retinanet_effb3_fpn_8xb4-crop896-1x_coco --dest ./checkpoints

# !mim download mmdet --config faster-rcnn_r50_fpn_1x_coco --dest ./checkpoints

# !mim download mmdet --config cascade-rcnn_r101_fpn_20e_coco --dest ./checkpoints
# !mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints


In [ ]:
%cd ..
print("\nCopy path below")
%cd data

In [ ]:
%cd ..
%cd mmdetection

## Make config file for training  
  
You must modify:  
  
(1) data_root  
(2) Model related paths: "_base_", "load_from", "with open"

following example is based on retinanet

In [ ]:
config_kitti = """
# Inherit and overwrite part of the config based on this config
_base_ = './retinanet_effb3_fpn_8xb4-crop896-1x_coco.py' 
data_root = 'Paste path above/' # dataset root, write down path (not relative) for your data directory

# Modify dataset related settings
train_batch_size_per_gpu = 4
train_num_workers = 2

# Modify model related settings
max_epochs = 50
stage2_num_epochs = 1
base_lr = 0.00008

metainfo = {
    'classes': ('Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc' ),
    'palette': [
        (220, 20, 60),    # Red
        (0, 0, 255),      # Blue
        (0, 255, 0),      # Green
        (255, 255, 0),    # Yellow
        (255, 165, 0),    # Orange
        (128, 0, 128),    # Purple
        (255, 0, 0),      # Crimson
        (0, 255, 255),    # Cyan
        (128, 128, 128),  # Gray
        (255, 192, 203),  # Pink
        (0, 128, 0),      # Dark Green
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/image/'),
        ann_file='train/coco/kitti_coco_format_train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='val/image/'),
        ann_file='val/coco/kitti_coco_format_val.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + 'val/coco/kitti_coco_format_val.json')

test_evaluator = val_evaluator

# learning rate
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0e-5,
        by_epoch=False,
        begin=0,
        end=10),
    dict(
        # use cosine lr from 10 to 20 epoch
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 2,
        end=max_epochs,
        T_max=max_epochs // 2,
        by_epoch=True,
        convert_to_iter_based=True),
]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=5))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight (copy and paste from model downloading cell)
load_from = 'checkpoints/retinanet_effb3_fpn_crop896_8x4_1x_coco_20220322_234806-615a0dda.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
"""

# save config to file
with open('configs/efficientnet/retinanet_effb3_fpn_8xb4-crop896-1x_kitti.py', 'w') as f:
    f.write(config_kitti)

## Training command

Recommended to run at terminal rather than at ipynb file

In [ ]:
!python tools/train.py configs/efficientnet/retinanet_effb3_fpn_8xb4-crop896-1x_kitti.py


## Config for test set with tracking dataset

Modify test dataloader and test evaluater in config file

(1) test dataloader

```
test_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='test/images/'),
        ann_file='test/coco/kitti_coco_format_test.json'))

```

(2) test evaluater

```
test_evaluator = dict(ann_file=data_root + 'test/coco/kitti_coco_format_test.json')

```

In [ ]:
config_kitti = """
# Inherit and overwrite part of the config based on this config
_base_ = './faster-rcnn_r50_fpn_1x_coco.py' 
data_root = 'Paste path above' # dataset root, write down path (not relative) for your data directory

# Modify dataset related settings
train_batch_size_per_gpu = 4
train_num_workers = 2

# Modify model related settings
max_epochs = 50
stage2_num_epochs = 1
base_lr = 0.00008

metainfo = {
    'classes': ('Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc' ),
    'palette': [
        (220, 20, 60),    # Red
        (0, 0, 255),      # Blue
        (0, 255, 0),      # Green
        (255, 255, 0),    # Yellow
        (255, 165, 0),    # Orange
        (128, 0, 128),    # Purple
        (255, 0, 0),      # Crimson
        (0, 255, 255),    # Cyan
        (128, 128, 128),  # Gray
        (255, 192, 203),  # Pink
        (0, 128, 0),      # Dark Green
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/image/'),
        ann_file='train/coco/kitti_coco_format_train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='val/image/'),
        ann_file='val/coco/kitti_coco_format_val.json'))

test_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='test/images/'),
        ann_file='test/coco/kitti_coco_format_test.json'))

val_evaluator = dict(ann_file=data_root + 'val/coco/kitti_coco_format_val.json')

test_evaluator = dict(ann_file=data_root + 'test/coco/kitti_coco_format_test.json')

# learning rate
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0e-5,
        by_epoch=False,
        begin=0,
        end=10),
    dict(
        # use cosine lr from 10 to 20 epoch
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 2,
        end=max_epochs,
        T_max=max_epochs // 2,
        by_epoch=True,
        convert_to_iter_based=True),
]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=5))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight (copy and paste from model downloading cell)
load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
"""

# save config to file
with open('configs/faster_rcnn/faster-rcnn_r50_fpn_1x_kitti.py', 'w') as f:
    f.write(config_kitti)